In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import json
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') #make sure list up to date
from nltk.corpus import wordnet
from IPython.display import clear_output
from scipy import stats

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
chartsDf = pd.read_csv('letras_top200.csv', sep = '|')
chartsDf.head()

,Track,Artist,Lyrics
0,Surtada,Dadá Boladão,"[Letra de ""Surtada (Remix)"" com Dadá Boladão, ..."
1,Gaiola É o Troco,MC Du Black,"Gaiola é o troco\nDu Black na voz, na voz, na ..."
2,Some que ele vem atrás,Anitta,Amiga do céu\nMeu boy me largou\nParou de segu...
3,Supera,Marília Mendonça,Tá de novo com essa pessoa\nNão tô acreditand...
4,Hit Contagiante,Felipe Original,"Ahn, fala tu que eu tô cansado\nFelipe Origina..."


# Get Lemmas and Drop Stopwords from Lyrics

In [3]:
def lineSplitAndClean(Lyrics): 
    '''return the lyrics split into lines and stripped of punctuation characters'''
    
        
        #splitting on \n instead of using sent_tokenize() from nltk as I don't
        #expect song lyrics to have great sentence structure so think \n will
        #be a more reliable delimiter
        #using .lower both for uniformity and cause stopwords list is all lowercase
    lines = [line.lower() for line in Lyrics.split('\n') if len(line)>0]
        
        #strip any punctuation
    lines = [re.sub(r'\W',' ',line) for line in lines]

        #replace any multiple consecutive spaces with just one space
    lines = [re.sub(r'\s+',' ',line) for line in lines]
        
        #possible for lines to be an empty list here like for instrumental
        #songs the starting lyrics will just be '[instrumental]'
    return lines

In [4]:
#breaking lyrics into lines so nltk.pos_tag get's sentences passed to it rather 
#than all the words at once. Should give better context for tagging.
chartsDf['lines'] = chartsDf.Lyrics.apply(lineSplitAndClean)

chartsDf.head(10)

,Track,Artist,Lyrics,lines
0,Surtada,Dadá Boladão,"[Letra de ""Surtada (Remix)"" com Dadá Boladão, ...",[ letra de surtada remix com dadá boladão tati...
1,Gaiola É o Troco,MC Du Black,"Gaiola é o troco\nDu Black na voz, na voz, na ...","[gaiola é o troco, du black na voz na voz na v..."
2,Some que ele vem atrás,Anitta,Amiga do céu\nMeu boy me largou\nParou de segu...,"[amiga do céu, meu boy me largou, parou de seg..."
3,Supera,Marília Mendonça,Tá de novo com essa pessoa\nNão tô acreditand...,"[tá de novo com essa pessoa, não tô acreditand..."
4,Hit Contagiante,Felipe Original,"Ahn, fala tu que eu tô cansado\nFelipe Origina...","[ahn fala tu que eu tô cansado, felipe origina..."
5,quando a bad bater,Luan Santana,"Amor, eu só quero seu bem\nSe for embora agora...","[amor eu só quero seu bem, se for embora agora..."
6,Tijolão,Jorge & Mateus,Eu te conheci depois que a gente largou\nSerá ...,"[eu te conheci depois que a gente largou, será..."
7,Ferida Curada,Zé Neto & Cristiano,"Tão cheia de certeza, você duvidou\nQue não ex...","[tão cheia de certeza você duvidou, que não ex..."
8,Invocada,Ludmilla,"Eu te amava, não posso negar, também sofria, n...",[eu te amava não posso negar também sofria não...
9,Lençol Dobrado,João Gustavo e Murilo,"Bora João Gustavo\nVai Murilo\n1, 2, 3\nVai, p...","[bora joão gustavo, vai murilo, 1 2 3, vai pra..."


In [7]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def getLemmasAndWordTags(lines):
    '''
    return a list of tuples of (lemma, word type tag) for each word
    in the passed lines
    '''
    
    wordsList = []
    
    for line in lines:
        words = nltk.word_tokenize(line, language='portuguese')
            
        if words not in nltk.corpus.stopwords.words('portuguese'): 
            wordsList.append(words)
  
    return wordsList

In [8]:
#lemmaSourceWords = {} #will be populated by getLemmasAndWordTags function
chartsDf['lemmas'] = chartsDf.lines.apply(getLemmasAndWordTags)
chartsDf.head(10)

,Track,Artist,Lyrics,lines,lemmas
0,Surtada,Dadá Boladão,"[Letra de ""Surtada (Remix)"" com Dadá Boladão, ...",[ letra de surtada remix com dadá boladão tati...,"[[letra, de, surtada, remix, com, dadá, boladã..."
1,Gaiola É o Troco,MC Du Black,"Gaiola é o troco\nDu Black na voz, na voz, na ...","[gaiola é o troco, du black na voz na voz na v...","[[gaiola, é, o, troco], [du, black, na, voz, n..."
2,Some que ele vem atrás,Anitta,Amiga do céu\nMeu boy me largou\nParou de segu...,"[amiga do céu, meu boy me largou, parou de seg...","[[amiga, do, céu], [meu, boy, me, largou], [pa..."
3,Supera,Marília Mendonça,Tá de novo com essa pessoa\nNão tô acreditand...,"[tá de novo com essa pessoa, não tô acreditand...","[[tá, de, novo, com, essa, pessoa], [não, tô, ..."
4,Hit Contagiante,Felipe Original,"Ahn, fala tu que eu tô cansado\nFelipe Origina...","[ahn fala tu que eu tô cansado, felipe origina...","[[ahn, fala, tu, que, eu, tô, cansado], [felip..."
5,quando a bad bater,Luan Santana,"Amor, eu só quero seu bem\nSe for embora agora...","[amor eu só quero seu bem, se for embora agora...","[[amor, eu, só, quero, seu, bem], [se, for, em..."
6,Tijolão,Jorge & Mateus,Eu te conheci depois que a gente largou\nSerá ...,"[eu te conheci depois que a gente largou, será...","[[eu, te, conheci, depois, que, a, gente, larg..."
7,Ferida Curada,Zé Neto & Cristiano,"Tão cheia de certeza, você duvidou\nQue não ex...","[tão cheia de certeza você duvidou, que não ex...","[[tão, cheia, de, certeza, você, duvidou], [qu..."
8,Invocada,Ludmilla,"Eu te amava, não posso negar, também sofria, n...",[eu te amava não posso negar também sofria não...,"[[eu, te, amava, não, posso, negar, também, so..."
9,Lençol Dobrado,João Gustavo e Murilo,"Bora João Gustavo\nVai Murilo\n1, 2, 3\nVai, p...","[bora joão gustavo, vai murilo, 1 2 3, vai pra...","[[bora, joão, gustavo], [vai, murilo], [1, 2, ..."


# Investigando recorrências

In [112]:
def countWords(x, word):

    count_lst = []

    for lst in x:
        for element in lst:
            count_lst.append(element.count(word))
        
    return sum(count_lst)

In [144]:
dict_amor = {}

amor = ['amo','amor', 'saudade', 'saudades', 'supera', 'carinho']

for item in amor:
    dict_amor[item] = (chartsDf.apply(lambda x: countWords(x['lemmas'], item), axis=1)).sum()
    
dict_amor

{'amo': 0       0
 1       4
 2       0
 3       2
 4       0
 5       4
 6       0
 7       5
 8       0
 9       7
 10     16
 11      0
 12      0
 13      0
 14      0
 15      0
 16     12
 17      1
 18      0
 19      1
 20      0
 21      6
 22      0
 23      0
 24      0
 25      0
 26      2
 27      2
 28      1
 29      4
        ..
 170     0
 171     4
 172     0
 173     4
 174     0
 175     0
 176     1
 177     2
 178     0
 179     0
 180     2
 181     0
 182     0
 183     2
 184     1
 185     2
 186     0
 187     0
 188     0
 189     2
 190     2
 191     0
 192     0
 193     2
 194     0
 195     4
 196     2
 197     0
 198     0
 199     0
 Length: 200, dtype: int64, 'amor': 0       0
 1       4
 2       0
 3       2
 4       0
 5       4
 6       0
 7       5
 8       0
 9       5
 10     10
 11      0
 12      0
 13      0
 14      0
 15      0
 16     11
 17      1
 18      0
 19      0
 20      0
 21      6
 22      0
 23      0
 24      0
 25      0
 

In [129]:
dict_bebidas = {}

bebidas = ['beber', 'bebo', 'bebida', 'cerveja', 'gelada', 'vinho', 'whisky', 'vodka', 'conhaque', 'tequila', 'cachaça',
           'gim', 'vinho', 'suco', 'água', 'hennessy', 'champanhe','licor']

for item in bebidas:
    dict_bebidas[item] = (chartsDf.apply(lambda x: countWords(x['lemmas'], item), axis=1)).sum()
    
dict_bebidas

{'beber': 38,
 'bebo': 9,
 'bebida': 16,
 'cerveja': 16,
 'gelada': 7,
 'vinho': 13,
 'whisky': 3,
 'vodka': 2,
 'conhaque': 3,
 'tequila': 4,
 'cachaça': 6,
 'gim': 3,
 'suco': 2,
 'água': 14,
 'hennessy': 2,
 'champanhe': 4,
 'licor': 1}

In [139]:
dict_carros = {}

carros = ['honda', 'kia', 'audi', 'dodge', 'lamborghini', 'porsche', 'mitsubishi']

for item in carros:
    dict_carros[item] = (chartsDf.apply(lambda x: countWords(x['lemmas'], item), axis=1)).sum()
    
dict_carros

{'honda': 4,
 'kia': 4,
 'audi': 5,
 'dodge': 2,
 'lamborghini': 1,
 'porsche': 2,
 'mitsubishi': 1}

In [143]:
dict_marcas = {}

marcas = ['nike', 'zara', 'rolex', 'gucci', 'versace', 'osklen']

for item in marcas:
    dict_marcas[item] = (chartsDf.apply(lambda x: countWords(x['lemmas'], item), axis=1)).sum()
    
dict_marcas

{'nike': 1, 'zara': 1, 'rolex': 1, 'gucci': 3, 'versace': 2, 'osklen': 0}